In [2]:
import time
import os
import pickle

from simtk.openmm import XmlSerializer
from simtk import openmm
from simtk.openmm import app, unit
import openmmtools

# Check computation time for setUseDispersionCorrection(True) for only the NonbondedForce that contains nonalchemical atoms, but not the CustomNonbondedForce that contains alchemical atoms

Use long-range env

In [2]:
with open('rbd_ace2_example/rbd_ace2_hybrid_system.xml', 'r') as f:
    system = XmlSerializer.deserialize(f.read())
with open('rbd_ace2_example/rbd_ace2_state.xml', 'r') as f:
    state = XmlSerializer.deserialize(f.read())

In [3]:
system.getForces()

[<openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2af9705bb930> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2af9705bb960> >,
 <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2af9705bba80> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2af9705bbd80> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2af9705bbde0> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2af9705bbb40> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2af9705bbe10> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2af9705bbf30> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of t

In [4]:
system.getForce(7).getUseDispersionCorrection()

True

In [5]:
system.getForce(8).setUseLongRangeCorrection(False)

In [6]:
# Set up integrator
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin, 
                                                        collision_rate=1 / unit.picoseconds, 
                                                        timestep=4 * unit.femtoseconds)


In [7]:
# Set up context
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
context.setPositions(state.getPositions())
context.setVelocitiesToTemperature(300 * unit.kelvin)

KeyboardInterrupt: 

In [16]:
nsteps = 100
initial_time = time.time()
for step in range(nsteps):
#     context.setParameter('lambda_sterics_insert', (step + 1) * 0.01)
    integrator.step(1)
elapsed_time = (time.time() - initial_time) * unit.seconds

OpenMMException: Error invoking kernel: CUDA_ERROR_ILLEGAL_ADDRESS (700)

In [ ]:
elapsed_time

# Check computation time for setLongRangeCorrection(True) for both CustomNonbondedForces

Use long-range env

In [14]:
with open('rbd_ace2_example/rbd_ace2_hybrid_system.xml', 'r') as f:
    system = XmlSerializer.deserialize(f.read())
with open('rbd_ace2_example/rbd_ace2_state.xml', 'r') as f:
    state = XmlSerializer.deserialize(f.read())

In [16]:
system.getForce(7).getUseDispersionCorrection()

True

In [15]:
system.getForce(8).setUseLongRangeCorrection(True)

In [17]:
# Set up integrator
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin, 
                                                        collision_rate=1 / unit.picoseconds, 
                                                        timestep=4 * unit.femtoseconds)


In [18]:
# Set up context
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
context.setPositions(state.getPositions())
context.setVelocitiesToTemperature(300 * unit.kelvin)

In [19]:
nsteps = 100
initial_time = time.time()
for step in range(nsteps):
    context.setParameter('lambda_sterics_insert', (step + 1) * 0.01)
    integrator.step(1)
elapsed_time = (time.time() - initial_time) * unit.seconds

In [20]:
elapsed_time

Quantity(value=36.05855202674866, unit=second)

# Check computation time for setLongRangeCorrection(True) for both CustomNonbondedForces (and with Peter's fix)

Use long-range-peter env

In [10]:
with open('rbd_ace2_example/rbd_ace2_hybrid_system.xml', 'r') as f:
    system = XmlSerializer.deserialize(f.read())
with open('rbd_ace2_example/rbd_ace2_state.xml', 'r') as f:
    state = XmlSerializer.deserialize(f.read())

In [11]:
system.getForce(7).getUseDispersionCorrection()

True

In [12]:
system.getForce(8).setUseLongRangeCorrection(True)

In [13]:
# Set up integrator
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin, 
                                                        collision_rate=1 / unit.picoseconds, 
                                                        timestep=4 * unit.femtoseconds)


In [14]:
# Set up context
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
context.setPositions(state.getPositions())
context.setVelocitiesToTemperature(300 * unit.kelvin)

In [15]:
nsteps = 100
initial_time = time.time()
for step in range(nsteps):
    context.setParameter('lambda_sterics_insert', (step + 1) * 0.01)
    integrator.step(1)
elapsed_time = (time.time() - initial_time) * unit.seconds

In [16]:
elapsed_time

Quantity(value=11.170037746429443, unit=second)

# Peter's test code

In [12]:
from openmm import *
import time
from openmmtools.integrators import LangevinIntegrator


In [13]:
import openmm

In [14]:
openmm.version.git_revision

'779400a1e3fdc988e84fa7aed760ac95d81c179c'

In [15]:
with open('rbd_ace2_hybrid_system.xml') as input:
    system = XmlSerializer.deserialize(input.read())
# with open('rbd_ace2_state.xml') as input:
#     state = XmlSerializer.deserialize(input.read())
with open('rbd_ace2_state_minimized2.xml') as input:
    state = XmlSerializer.deserialize(input.read())

In [18]:
integrator = LangevinMiddleIntegrator(300, 1.0, 0.004)
# integrator = LangevinIntegrator(temperature=300 * unit.kelvin, 
#                                                         collision_rate=1 / unit.picoseconds, 
#                                                         timestep=4 * unit.femtoseconds)
platform = Platform.getPlatformByName('CUDA')
context = Context(system, integrator, platform)
context.setState(state)


In [19]:
# LocalEnergyMinimizer.minimize(context)
# state = context.getState(getPositions=True, getVelocities=True)
# with open('rbd_ace2_state_minimized2.xml', 'w') as f:
#     f.write(XmlSerializer.serialize(state))


In [20]:
integrator.step(10)
context.getState(getPositions=True)
start = time.time()
for i in range(100):
    integrator.step(1)
    context.setParameter('lambda_sterics_insert', 0.1*(i%2))
context.getState(getPositions=True)
end = time.time()
print(end-start)

1.4290649890899658


In [17]:
integrator.step(10)
context.getState(getPositions=True)
start = time.time()
for i in range(100):
    integrator.step(1)
#     context.setParameter('lambda_sterics_insert', 0.1*(i%2))
context.getState(getPositions=True)
end = time.time()
print(end-start)

0.6654658317565918


In [19]:
system.getForces()

[<openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2aef466dbc30> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2aef466db4e0> >,
 <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2aef466db660> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2aef45c87150> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2aef45c873c0> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2aef45c871e0> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2aef45cb38d0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2aef45cb3e40> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of t

In [20]:
system.getForce(7).getUseDispersionCorrection()

True

In [23]:
system.getForce(8).getUseLongRangeCorrection()

True

# Peter's test code (with long-range) -- no PR fixes

In [3]:
from openmm import *
import time
from openmmtools.integrators import LangevinIntegrator


In [4]:
import openmm

In [5]:
openmm.version.git_revision

'ad113a0cb37991a2de67a08026cf3b91616bafbe'

In [6]:
with open('rbd_ace2_hybrid_system.xml') as input:
    system = XmlSerializer.deserialize(input.read())
# with open('rbd_ace2_state.xml') as input:
#     state = XmlSerializer.deserialize(input.read())
with open('rbd_ace2_state_minimized2.xml') as input:
    state = XmlSerializer.deserialize(input.read())

In [9]:
integrator = LangevinMiddleIntegrator(300, 1.0, 0.004)
# integrator = LangevinIntegrator(temperature=300 * unit.kelvin, 
#                                                         collision_rate=1 / unit.picoseconds, 
#                                                         timestep=4 * unit.femtoseconds)
platform = Platform.getPlatformByName('CUDA')
context = Context(system, integrator, platform)
context.setState(state)


In [10]:
integrator.step(10)
context.getState(getPositions=True)
start = time.time()
for i in range(100):
    integrator.step(1)
    context.setParameter('lambda_sterics_insert', 0.1*(i%2))
context.getState(getPositions=True)
end = time.time()
print(end-start)

22.0093994140625


In [8]:
integrator.step(10)
context.getState(getPositions=True)
start = time.time()
for i in range(100):
    integrator.step(1)
#     context.setParameter('lambda_sterics_insert', 0.1*(i%2))
context.getState(getPositions=True)
end = time.time()
print(end-start)

0.6648898124694824


# Peter's test code (with long-range) -- PR # 3275 fixes

In [1]:
from openmm import *
import time
from openmmtools.integrators import LangevinIntegrator


In [2]:
import openmm

In [3]:
openmm.version.git_revision

'54c9ceb902a202d919cb19005813218f3f4e9bf1'

In [4]:
with open('rbd_ace2_hybrid_system.xml') as input:
    system = XmlSerializer.deserialize(input.read())
# with open('rbd_ace2_state.xml') as input:
#     state = XmlSerializer.deserialize(input.read())
with open('rbd_ace2_state_minimized2.xml') as input:
    state = XmlSerializer.deserialize(input.read())

In [5]:
integrator = LangevinMiddleIntegrator(300, 1.0, 0.004)
# integrator = LangevinIntegrator(temperature=300 * unit.kelvin, 
#                                                         collision_rate=1 / unit.picoseconds, 
#                                                         timestep=4 * unit.femtoseconds)
platform = Platform.getPlatformByName('CUDA')
context = Context(system, integrator, platform)
context.setState(state)


In [6]:
integrator.step(10)
context.getState(getPositions=True)
start = time.time()
for i in range(100):
    integrator.step(1)
    context.setParameter('lambda_sterics_insert', 0.1*(i%2))
context.getState(getPositions=True)
end = time.time()
print(end-start)

0.8623063564300537


In [6]:
integrator.step(10)
context.getState(getPositions=True)
start = time.time()
for i in range(100):
    integrator.step(1)
#     context.setParameter('lambda_sterics_insert', 0.1*(i%2))
context.getState(getPositions=True)
end = time.time()
print(end-start)

0.7842466831207275


# Test with PeriodicNonequilibriumIntegrator (without long range PRs)

In [2]:
from openmm import *
import time
from openmmtools.integrators import PeriodicNonequilibriumIntegrator, LangevinIntegrator

In [3]:
import openmm

In [4]:
openmm.version.git_revision

'ad113a0cb37991a2de67a08026cf3b91616bafbe'

In [5]:
with open('rbd_ace2_hybrid_system.xml') as input:
    system = XmlSerializer.deserialize(input.read())
# with open('rbd_ace2_state.xml') as input:
#     state = XmlSerializer.deserialize(input.read())
with open('rbd_ace2_state_minimized2.xml') as input:
    state = XmlSerializer.deserialize(input.read())

In [7]:
system.getForce(7).getUseDispersionCorrection()

True

In [8]:
system.getForce(8).getUseLongRangeCorrection()

True

In [9]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                              'lambda_sterics_core': x,
                              'lambda_electrostatics_core': x,
                              'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                              'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                              'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                              'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                              'lambda_bonds': x,
                              'lambda_angles': x,
                              'lambda_torsions': x}

# Define simulation parameters
nsteps_eq = 10
nsteps_neq = 100
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
temperature = 300.0 * unit.kelvin

integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)

platform = Platform.getPlatformByName('CUDA')
context = Context(system, integrator, platform)
context.setState(state)


In [10]:
integrator.step(10) # eq

start = time.time()
for _ in range(10):
    integrator.step(10)
    work = integrator.get_protocol_work(dimensionless=True)
end = time.time()
print(end-start)

22.67329239845276


In [12]:
# Set up integrator
integrator = LangevinIntegrator(temperature=300 * unit.kelvin, 
                                                        collision_rate=1 / unit.picoseconds, 
                                                        timestep=4 * unit.femtoseconds)

platform = Platform.getPlatformByName('CUDA')
context = Context(system, integrator, platform)
context.setState(state)


In [13]:
integrator.step(10) # eq

start = time.time()
for _ in range(10):
    integrator.step(10)
end = time.time()
print(end-start)

0.7495777606964111


# Test with PeriodicNonequilibriumIntegrator (with 2nd PR #3275) -- note this includes 1st PR too

In [ ]:
from openmm import *
import time
from openmmtools.integrators import PeriodicNonequilibriumIntegrator

In [ ]:
import openmm

In [ ]:
openmm.version.git_revision

In [9]:
with open('rbd_ace2_hybrid_system.xml') as input:
    system = XmlSerializer.deserialize(input.read())
# with open('rbd_ace2_state.xml') as input:
#     state = XmlSerializer.deserialize(input.read())
with open('rbd_ace2_state_minimized2.xml') as input:
    state = XmlSerializer.deserialize(input.read())

In [10]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                              'lambda_sterics_core': x,
                              'lambda_electrostatics_core': x,
                              'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                              'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                              'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                              'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                              'lambda_bonds': x,
                              'lambda_angles': x,
                              'lambda_torsions': x}

# Define simulation parameters
nsteps_eq = 10
nsteps_neq = 100
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
temperature = 300.0 * unit.kelvin

integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)

platform = Platform.getPlatformByName('CUDA')
context = Context(system, integrator, platform)
context.setState(state)


In [11]:
integrator.step(10) # eq

start = time.time()
for _ in range(10):
    integrator.step(10)
    work = integrator.get_protocol_work(dimensionless=True)
end = time.time()
print(end-start)

2.195697546005249


# Test with PeriodicNonequilibriumIntegrator (with 1st PR #3236)

In [6]:
from openmm import *
import time
from openmmtools.integrators import PeriodicNonequilibriumIntegrator

In [7]:
import openmm

In [8]:
openmm.version.git_revision

'779400a1e3fdc988e84fa7aed760ac95d81c179c'

In [9]:
with open('rbd_ace2_hybrid_system.xml') as input:
    system = XmlSerializer.deserialize(input.read())
# with open('rbd_ace2_state.xml') as input:
#     state = XmlSerializer.deserialize(input.read())
with open('rbd_ace2_state_minimized2.xml') as input:
    state = XmlSerializer.deserialize(input.read())

In [10]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                              'lambda_sterics_core': x,
                              'lambda_electrostatics_core': x,
                              'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                              'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                              'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                              'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                              'lambda_bonds': x,
                              'lambda_angles': x,
                              'lambda_torsions': x}

# Define simulation parameters
nsteps_eq = 10
nsteps_neq = 100
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
temperature = 300.0 * unit.kelvin

integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)

platform = Platform.getPlatformByName('CUDA')
context = Context(system, integrator, platform)
context.setState(state)


In [11]:
integrator.step(10) # eq

start = time.time()
for _ in range(10):
    integrator.step(10)
    work = integrator.get_protocol_work(dimensionless=True)
end = time.time()
print(end-start)

2.154646873474121


# Test ala dipeptide in solvent

In [4]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmm import *
import time



In [2]:
atp, system_generator = generate_atp(phase='solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology,
                                     new_res='THR',
                                     system=atp.system,
                                     positions=atp.positions,
                                     system_generator=system_generator,
                                     conduct_htf_prop=True,
                                     validate_endstate_energy=False)
system = htf.hybrid_system

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]


making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 18, 14, 13, 15, 17, 16, 19]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1645 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

INFO:geometry:total reduced potential before atom placement: 9.116481141712038


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.67298668767817
INFO:geometry:final reduced energy 29.78946596595155
INFO:geometry:sum of energies: 29.789467829390205
INFO:geometry:magnitude of difference in the energies: 1.8634386549365445e-06
INFO:geometry:Final logp_proposal: -26804.085411836873


added energy components: [('CustomBondForce', 0.0005202039273265048), ('CustomAngleForce', 0.45111977562481365), ('CustomTorsionForce', 7.2504633440858095), ('CustomBondForce', 12.970883364040219)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.56477354), unit=nanometer)]
INFO:relati

In [17]:
integrator = LangevinMiddleIntegrator(300, 1.0, 0.004)
platform = Platform.getPlatformByName('CUDA')
context = Context(system, integrator, platform)
context.setPositions(htf.hybrid_positions)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())


In [18]:
LocalEnergyMinimizer.minimize(context)


In [7]:
for k, v in context.getParameters().items():
    print(k, v)

MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_angles 0.0
lambda_bonds 0.0
lambda_electrostatics_core 0.0
lambda_electrostatics_delete 0.0
lambda_electrostatics_insert 0.0
lambda_sterics_core 0.0
lambda_sterics_delete 0.0
lambda_sterics_insert 0.0
lambda_torsions 0.0
softcore_alpha 0.85


In [8]:
state = context.getState(getPositions=True, getVelocities=True)
with open('ala_dipeptide_solvent_state_minimized.xml', 'w') as f:
    f.write(XmlSerializer.serialize(state))

with open('ala_dipeptide_solvent_system.xml', 'w') as f:
    f.write(XmlSerializer.serialize(system))


In [13]:
integrator.step(10)
context.getState(getPositions=True)
start = time.time()
for i in range(100):
#     print(i)
    integrator.step(1)
    context.setParameter('lambda_sterics_insert', 0.1*(i%2))
context.getState(getPositions=True)
end = time.time()
print(end-start)

0.25411224365234375


In [19]:
integrator.step(10)
context.getState(getPositions=True)
start = time.time()
for i in range(100):
#     print(i)
    integrator.step(1)
#     context.setParameter('lambda_sterics_insert', 0.1*(i%2))
context.getState(getPositions=True)
end = time.time()
print(end-start)

0.18193483352661133
